download the weights using

https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5

# Without Using Dropout

In [1]:
import os

from tensorflow.keras import layers, Model

from tensorflow.keras.applications.inception_v3 import InceptionV3


local_weight_file = ('inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5')

pre_trained_model = InceptionV3(input_shape = (150,150,3),
                               include_top = False,
                               weights =None)

pre_trained_model.load_weights(local_weight_file)

# pre_trained_model.summary()

for layer in pre_trained_model.layers:
    layer.trainable = False
    
last_layer = pre_trained_model.get_layer('mixed7')
print("last layer output shape: ", last_layer.output_shape)
last_output = last_layer.output

last layer output shape:  (None, 7, 7, 768)


In [2]:
from tensorflow.keras.optimizers import RMSprop

# Flatten the output layer to 1 dimension
x = layers.Flatten()(last_output)

# Add a fully connected layer with 1,024 hidden units and ReLU activation
x = layers.Dense(1024, activation='relu')(x)

# Add a final sigmoid layer for classification
x = layers.Dense(1, activation='sigmoid')(x)

model = Model(pre_trained_model.input, x)

model.compile(optimizer =RMSprop(lr =0.0001),
             loss= 'binary_crossentropy',
             metrics = ['accuracy'])


In [3]:
#model.summary()

In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import os
import zipfile

local_zip = 'cats_and_dogs_filtered.zip'

zip_ref = zipfile.ZipFile(local_zip, 'r')

zip_ref.extractall('/tmp')
zip_ref.close()

# Define our example directories and files
base_dir = '/tmp/cats_and_dogs_filtered'

train_dir = os.path.join( base_dir, 'train')
validation_dir = os.path.join( base_dir, 'validation')


train_cats_dir = os.path.join(train_dir, 'cats') # Directory with our training cat pictures
train_dogs_dir = os.path.join(train_dir, 'dogs') # Directory with our training dog pictures
validation_cats_dir = os.path.join(validation_dir, 'cats') # Directory with our validation cat pictures
validation_dogs_dir = os.path.join(validation_dir, 'dogs')# Directory with our validation dog pictures

train_cat_fnames = os.listdir(train_cats_dir)
train_dog_fnames = os.listdir(train_dogs_dir)

# Add our data-augmentation parameters to ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255.,
                                   rotation_range = 40,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

# Note that the validation data should not be augmented!
test_datagen = ImageDataGenerator( rescale = 1.0/255. )

# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size = 20,
                                                    class_mode = 'binary', 
                                                    target_size = (150, 150))     

# Flow validation images in batches of 20 using test_datagen generator
validation_generator =  test_datagen.flow_from_directory( validation_dir,
                                                          batch_size  = 20,
                                                          class_mode  = 'binary', 
                                                          target_size = (150, 150))

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [5]:
history = model.fit(
            train_generator,
            validation_data = validation_generator,
            steps_per_epoch = 100,
            epochs = 20,
            validation_steps = 50,
            verbose = 2)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 100 steps, validate for 50 steps
Epoch 1/20
100/100 - 68s - loss: 0.4931 - accuracy: 0.7710 - val_loss: 0.2686 - val_accuracy: 0.9100
Epoch 2/20
100/100 - 67s - loss: 0.3580 - accuracy: 0.8435 - val_loss: 0.2607 - val_accuracy: 0.9330
Epoch 3/20
100/100 - 67s - loss: 0.3328 - accuracy: 0.8555 - val_loss: 0.2472 - val_accuracy: 0.9450
Epoch 4/20
100/100 - 67s - loss: 0.3019 - accuracy: 0.8720 - val_loss: 0.6017 - val_accuracy: 0.9170
Epoch 5/20
100/100 - 67s - loss: 0.2722 - accuracy: 0.8855 - val_loss: 0.3403 - val_accuracy: 0.9460
Epoch 6/20
100/100 - 68s - loss: 0.2838 - accuracy: 0.8805 - val_loss: 0.6272 - val_accuracy: 0.9280
Epoch 7/20
100/100 - 67s - loss: 0.2845 - accuracy: 0.8830 - val_loss: 0.3389 - val_accuracy: 0.9540
Epoch 8/20
100/100 - 67s - loss: 0.2632 - accuracy: 0.8965 - val_loss: 0.3633 - val_accuracy: 0.9540
Epoch 9/20
100/100 - 67s - loss: 0.2496 - accuracy: 0.8915 - val_loss: 0.3297 - val_accuracy: 0.961

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()


plt.show()

# With Dropout

In [7]:
import os

from tensorflow.keras import layers, Model

from tensorflow.keras.applications.inception_v3 import InceptionV3


local_weight_file = ('inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5')

pre_trained_model = InceptionV3(input_shape = (150,150,3),
                               include_top = False,
                               weights =None)

pre_trained_model.load_weights(local_weight_file)
# pre_trained_model.summary()

for layer in pre_trained_model.layers:
    layer.trainable = False
    
last_layer = pre_trained_model.get_layer('mixed7')
print("last layer output shape: ", last_layer.output_shape)
last_output = last_layer.output

last layer output shape:  (None, 7, 7, 768)


In [8]:
from tensorflow.keras.optimizers import RMSprop

# Flatten the output layer to 1 dimension
x = layers.Flatten()(last_output)

# Add a fully connected layer with 1,024 hidden units and ReLU activation
x = layers.Dense(1024, activation='relu')(x)

# Add a dropout rate of 0.2
x = layers.Dropout(0.2)(x)

# Add a final sigmoid layer for classification
x = layers.Dense(1, activation='sigmoid')(x)

model = Model(pre_trained_model.input, x)

model.compile(optimizer =RMSprop(lr =0.0001),
             loss= 'binary_crossentropy',
             metrics = ['accuracy'])



In [9]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import os
import zipfile

local_zip = 'cats_and_dogs_filtered.zip'

zip_ref = zipfile.ZipFile(local_zip, 'r')

zip_ref.extractall('/tmp')
zip_ref.close()

# Define our example directories and files
base_dir = '/tmp/cats_and_dogs_filtered'

train_dir = os.path.join( base_dir, 'train')
validation_dir = os.path.join( base_dir, 'validation')


train_cats_dir = os.path.join(train_dir, 'cats') # Directory with our training cat pictures
train_dogs_dir = os.path.join(train_dir, 'dogs') # Directory with our training dog pictures
validation_cats_dir = os.path.join(validation_dir, 'cats') # Directory with our validation cat pictures
validation_dogs_dir = os.path.join(validation_dir, 'dogs')# Directory with our validation dog pictures

train_cat_fnames = os.listdir(train_cats_dir)
train_dog_fnames = os.listdir(train_dogs_dir)

# Add our data-augmentation parameters to ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255.,
                                   rotation_range = 40,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

# Note that the validation data should not be augmented!
test_datagen = ImageDataGenerator( rescale = 1.0/255. )

# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size = 20,
                                                    class_mode = 'binary', 
                                                    target_size = (150, 150))     

# Flow validation images in batches of 20 using test_datagen generator
validation_generator =  test_datagen.flow_from_directory( validation_dir,
                                                          batch_size  = 20,
                                                          class_mode  = 'binary', 
                                                          target_size = (150, 150))

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [ ]:
history = model.fit(
            train_generator,
            validation_data = validation_generator,
            steps_per_epoch = 100,
            epochs = 20,
            validation_steps = 50,
            verbose = 2)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 100 steps, validate for 50 steps
Epoch 1/20
100/100 - 72s - loss: 0.5173 - accuracy: 0.7645 - val_loss: 0.2412 - val_accuracy: 0.9260
Epoch 2/20
100/100 - 70s - loss: 0.3759 - accuracy: 0.8295 - val_loss: 0.2465 - val_accuracy: 0.9350
Epoch 3/20
100/100 - 73s - loss: 0.3456 - accuracy: 0.8410 - val_loss: 0.2787 - val_accuracy: 0.9370
Epoch 4/20
100/100 - 72s - loss: 0.3295 - accuracy: 0.8525 - val_loss: 0.2582 - val_accuracy: 0.9560
Epoch 5/20
100/100 - 70s - loss: 0.2948 - accuracy: 0.8715 - val_loss: 0.3334 - val_accuracy: 0.9430
Epoch 6/20
100/100 - 69s - loss: 0.3176 - accuracy: 0.8665 - val_loss: 0.3211 - val_accuracy: 0.9460
Epoch 7/20
100/100 - 71s - loss: 0.2874 - accuracy: 0.8820 - val_loss: 0.3348 - val_accuracy: 0.9490
Epoch 8/20
100/100 - 69s - loss: 0.3008 - accuracy: 0.8810 - val_loss: 0.2996 - val_accuracy: 0.9520
Epoch 9/20
100/100 - 71s - loss: 0.2663 - accuracy: 0.8940 - val_loss: 0.4419 - val_accuracy: 0.942

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()


plt.show()